In [1]:
import pandas as pd
import duckdb
pd.set_option('display.max_columns', 45)

# Read Data, init stat lists

In [2]:
with open("./data/queries/player_feature_eng.sql", "r") as f:
    sql = f.read()
con = duckdb.connect("data/nfl.duckdb")
df = con.execute(sql).fetchdf()

In [3]:
context_cols = ['player_id', 'player_display_name','position',
]
game_context = ['week','season_type', 'season', 'opponent_team', 'recent_team', 'game_id', 'game_type']
stat_cols = ['completions', 'attempts',
       'passing_yards', 'passing_tds', 'passing_interceptions', 'sacks_suffered', 'sack_yards_lost',
       'sack_fumbles', 'sack_fumbles_lost', 'passing_air_yards',
       'passing_yards_after_catch', 'passing_first_downs', 'passing_epa',
       'passing_2pt_conversions', 'pacr', 'carries', 'rushing_yards',
       'rushing_tds', 'rushing_fumbles', 'rushing_fumbles_lost',
       'rushing_first_downs', 'rushing_epa', 'rushing_2pt_conversions',
       'receptions', 'targets', 'receiving_yards', 'receiving_tds',
       'receiving_fumbles', 'receiving_fumbles_lost', 'receiving_air_yards',
       'receiving_yards_after_catch', 'receiving_first_downs', 'receiving_epa',
       'receiving_2pt_conversions', 'racr', 'target_share', 'air_yards_share',
       'wopr', 'special_teams_tds', 'fantasy_points', 'fantasy_points_ppr',]
rush_stat_cols = [
     'carries', 'rushing_yards',
       'rushing_tds', 'rushing_fumbles', 'rushing_fumbles_lost',
       'rushing_first_downs', 'rushing_epa', 'rushing_2pt_conversions',
       'receptions', 'targets', 'receiving_yards', 'receiving_tds',
       'fantasy_points', 'fantasy_points_ppr',
       'offense_snaps','offense_pct'
]

depth_cols = ['season', 'last_name',
       'first_name', 'football_name', 'formation', 'gsis_id', 'jersey_number',
       'position', 'elias_id', 'depth_position', 'player_display_name']


qb_stats_eng = [
    "completions",
    "attempts",
    "passing_yards",
    "passing_tds",
    "passing_interceptions",
    "sacks_suffered",
    "sack_yards_lost",
    "passing_air_yards",
    "passing_yards_after_catch",
    "passing_first_downs",
    "passing_2pt_conversions",
    "carries",
    "fantasy_points_ppr",
       'offense_snaps','offense_pct',
       #"completion_pct"
]

rush_stats_eng = [
    "carries",
    "rushing_yards",
    "rushing_tds",
    "rushing_first_downs",
    "receptions",
    "targets",
    "receiving_yards",
    "receiving_tds",
    "fantasy_points",
    "fantasy_points_ppr",
       'offense_snaps','offense_pct',
      # 'yards_per_carry',
]
receiver_stats_eng = [
    "receptions",
    "targets",
    "receiving_yards",
    "receiving_tds",
    "receiving_air_yards",
    "receiving_yards_after_catch",
    "receiving_first_downs",
    "target_share",
    "air_yards_share",
    "fantasy_points_ppr",
      'offense_snaps','offense_pct'
]
stats_to_eng = list(set(qb_stats_eng + rush_stats_eng + receiver_stats_eng))


# Rolling Averages

In [4]:
df.loc[~df.offense_snaps.isna(), stat_cols] = df.loc[~df.offense_snaps.isna(), stat_cols].fillna(0)
roll_columns = [x + '_roll' for x in stats_to_eng]
df[roll_columns] = df.groupby(context_cols)[stats_to_eng].transform(lambda x: x.ewm(span=7, min_periods=0).mean())
df[roll_columns] = df.groupby(context_cols)[roll_columns].shift(1)

### Compute efficiency Stats

In [5]:
def normalize_with_prev_league_avg(
    df: pd.DataFrame,
    new_col_name: str,
    numerator: str,
    denominator: str,
    padding: float | int,
) -> pd.DataFrame:
	"""Takes a rate stat and normalizes to league average by weighting with the league average
	for the prior season(to avoid data leakage)"""
	avg_df = df.groupby(["season"], as_index=False)[[numerator, denominator]].sum()
	avg_df["avg"] = (avg_df[numerator] / (1 + avg_df[denominator])).shift(1).bfill()
	new =  pd.merge(df, avg_df[["season", "avg"]], on="season")
	new[new_col_name] = ((padding * new['avg']) + new[numerator]) / (padding + new[denominator])
	return new.drop('avg',axis=1)
    #return pd.DataFrame()

In [6]:
rate_stat_args = [
	['completion_pct_roll', 'completions_roll','attempts_roll', 5],
	['yards_per_completion_roll', 'passing_yards_roll','completions_roll', 5],
	['yards_per_carry_roll', 'rushing_yards_roll','carries_roll', 5],
	['yards_per_catch_roll', 'receiving_yards_roll','receptions_roll', 5],
	['yards_per_target_roll', 'receiving_yards_roll','targets_roll', 5],
]

In [7]:
agg = df.copy()
for stat in rate_stat_args:
	agg = normalize_with_prev_league_avg(agg, *stat)

receiver_stats_eng += ['yards_per_catch','yards_per_target']
rush_stats_eng += ['yards_per_carry']
qb_stats_eng += ['completion_pct', 'yards_per_completion']

In [8]:
agg = agg[[x for x in agg.columns if x not in stat_cols]]
agg.to_parquet("data/agg/player_weekly_agg.parquet")
con.execute("CREATE OR REPLACE TABLE player_weekly_agg AS SELECT * FROM agg")

# Stack team stats

In [9]:
def stack_players(
    df: pd.DataFrame, position: list[str],  num_players: int, index_cols: list[str], value_cols: list[str]
) -> pd.DataFrame:
    
    df = df.loc[(df.dense_depth < num_players) & (~df.week.isna()) & (df.position.isin(position))]
    piv = df.pivot(
        index=index_cols, columns=["position", "dense_depth"], values=value_cols
    ).reset_index()
    piv.columns = [
        x[0] + "_" + x[1] + "_" + (str(int(x[2]))) if x[2] != "" else x[0] + x[1]
        for x in piv.columns
    ]
    return piv


In [10]:
player_cols = ["gsis_id", "player_display_name"]
qb_roll_stat_cols = player_cols + [x + "_roll" for x in qb_stats_eng]
receiver_roll_stat_cols = player_cols + [x + '_roll' for x in receiver_stats_eng] 
rush_stat_cols = player_cols + [x + '_roll' for x in rush_stats_eng]


team_qb_stats_raw = stack_players(agg, ['QB'],2, ["team", "season", "week"],qb_roll_stat_cols)
wr_temp = stack_players(agg, ['WR'] ,5,  ['team','season','week'], receiver_roll_stat_cols)
te_temp = stack_players(agg, ['TE'], 3,  ['team','season','week'], receiver_roll_stat_cols)
rb_temp =stack_players(agg, ['RB'],2,  ['team','season','week'], receiver_roll_stat_cols)
team_receiver_stats_raw = wr_temp.merge(te_temp, on=['team','season','week'])
team_receiver_stats_raw = team_receiver_stats_raw.merge(rb_temp, on=['team','season','week'])
team_rb_stats_raw = stack_players(agg, ['RB'] ,3,  ['team','season','week'], rush_stat_cols)

In [11]:
team_qb_stats_raw.to_csv('data/team_qb_stats.csv')
team_receiver_stats_raw.to_csv('data/team_receiver_stats.csv')
team_rb_stats_raw.to_csv('data/team_rb_stats.csv')

con.execute("CREATE OR REPLACE TABLE team_qb_stats AS SELECT * FROM team_qb_stats_raw")
con.execute("CREATE OR REPLACE TABLE team_receiver_stats AS SELECT * FROM team_receiver_stats_raw")
con.execute("CREATE OR REPLACE TABLE team_rb_stats AS SELECT * FROM team_rb_stats_raw")

In [12]:
con.close()